In [7]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol with an attacker, to demonstrate that the attacker can be detected.

In [ ]:
def prepare_singlet(qc, a, b):
    qc.x(b)
    qc.h(a)
    qc.cx(a, b)
    qc.z(a)

def measure_in_basis(qc, qubit, cbit, basis):
    if basis == 'Z':
        pass
    elif basis == 'X':
        qc.h(qubit)
    elif basis == 'W':
        qc.ry(-math.pi/4, qubit) # (Z+X)/sqr(2)
    elif basis == 'V':
        qc.ry(+math.pi/4, qubit) # (Z-X)/sqr(2)
    else:
        raise ValueError("basis must be one of 'Z','X','W','V'")
    qc.measure(qubit, cbit)

def bit_to_pm1(bit):
    return 1 if bit == 0 else -1

THETA_1_OVER_3 = 2 * math.acos(math.sqrt(1/3))

def sample_trit_1_over_3():
    backend = BasicSimulator()
    qc = QuantumCircuit(2, 2)
    qc.ry(THETA_1_OVER_3, 0)
    qc.measure(0, 0)
    qc.h(1)
    qc.measure(1, 1)
    tqc = transpile(qc, backend)
    result = backend.run(tqc, shots=1).result()
    bitstr = list(result.get_counts().keys())[0]
    c1 = int(bitstr[0]); c0 = int(bitstr[1])
    if c0 == 0:
        return 0
    return 1 if c1 == 0 else 2

ALICE_BASES = ['X', 'W', 'Z']
BOB_BASES   = ['W', 'Z', 'V']

def sample_alice_basis():
    return ALICE_BASES[sample_trit_1_over_3()]

def sample_bob_basis():
    return BOB_BASES[sample_trit_1_over_3()]

In [ ]:
def eve_dephase_bob_Z(qc, bob_qubit, eve_ancilla):
    qc.cx(bob_qubit, eve_ancilla)

def run_one_round_attacker():
    a_basis = sample_alice_basis()
    b_basis = sample_bob_basis()

    backend = BasicSimulator()
    # 3 qubits: q0 Alice, q1 Bob, q2 Eve ancilla; 2 cbits for A/B results
    qc = QuantumCircuit(3, 2)
    prepare_singlet(qc, 0, 1)

    # Eve attacks BEFORE Bob measures
    eve_dephase_bob_Z(qc, bob_qubit=1, eve_ancilla=2)

    measure_in_basis(qc, 0, 0, a_basis)
    measure_in_basis(qc, 1, 1, b_basis)

    tqc = transpile(qc, backend)
    result = backend.run(tqc, shots=1).result()
    bitstr = list(result.get_counts().keys())[0]
    b_bit = int(bitstr[0])
    a_bit = int(bitstr[1])
    return a_basis, b_basis, a_bit, b_bit

In [11]:
def qber(key_a, key_b):
    if len(key_a) == 0:
        return float('nan')
    mismatches = sum(1 for a,b in zip(key_a, key_b) if a != b)
    return mismatches / len(key_a)

def e91_attacker(N=80, max_rounds=None, verbose=True):
    if max_rounds is None:
        max_rounds = int(math.ceil(9*N/2 * 2))

    key_alice, key_bob = [], []
    sums = {'XW':0, 'XV':0, 'ZW':0, 'ZV':0}
    cnts = {'XW':0, 'XV':0, 'ZW':0, 'ZV':0}
    discards = 0

    rounds = 0
    while len(key_alice) < N and rounds < max_rounds:
        rounds += 1
        a_basis, b_basis, a_bit, b_bit = run_one_round_attacker()

        # key cases
        if a_basis == 'W' and b_basis == 'W':
            key_alice.append(a_bit)
            key_bob.append(1 - b_bit)
            continue
        if a_basis == 'Z' and b_basis == 'Z':
            key_alice.append(a_bit)
            key_bob.append(1 - b_bit)
            continue

        # CHSH cases
        prod = bit_to_pm1(a_bit) * bit_to_pm1(b_bit)
        if a_basis == 'X' and b_basis == 'W':
            sums['XW'] += prod; cnts['XW'] += 1
        elif a_basis == 'X' and b_basis == 'V':
            sums['XV'] += prod; cnts['XV'] += 1
        elif a_basis == 'Z' and b_basis == 'W':
            sums['ZW'] += prod; cnts['ZW'] += 1
        elif a_basis == 'Z' and b_basis == 'V':
            sums['ZV'] += prod; cnts['ZV'] += 1
        else:
            discards += 1

    def E(tag):
        return sums[tag] / cnts[tag] if cnts[tag] > 0 else float('nan')

    EXW, EXV, EZW, EZV = E('XW'), E('XV'), E('ZW'), E('ZV')
    S = abs(EXW - EXV + EZW + EZV)
    ok_key = (key_alice == key_bob)
    Q = qber(key_alice, key_bob)

    out = {
        'N_target': N,
        'rounds': rounds,
        'key_len': len(key_alice),
        'key_match': ok_key,
        'QBER': Q,
        'E': {'XW': EXW, 'XV': EXV, 'ZW': EZW, 'ZV': EZV},
        'S': S,
        'counts': cnts,
        'discards': discards,
        'key_alice': key_alice,
        'key_bob': key_bob,
    }

    if verbose:
        print(f"Rounds used: {rounds}")
        print(f"Key length: {len(key_alice)} (target {N})")
        print(f"Key matches after Bob flip? {ok_key}")
        print("QBER (after Bob flip) =", Q)
        print("E values:", out['E'])
        print("S =", S)
        print("CHSH sample counts:", cnts, " discards:", discards)

    return out

In [12]:
_ = e91_attacker(N=80, verbose=True)

Rounds used: 367
Key length: 80 (target 80)
Key matches after Bob flip? False
QBER (after Bob flip) = 0.1625
E values: {'XW': 0.15789473684210525, 'XV': 0.2222222222222222, 'ZW': -0.5, 'ZV': -0.8222222222222222}
S = 1.3865497076023392
CHSH sample counts: {'XW': 38, 'XV': 36, 'ZW': 32, 'ZV': 45}  discards: 136


The reduced value of S shows that the entanglement has been disrupted by the attacker.